In [1]:
# Check GPU configuration
!nvidia-smi

Sat Mar 22 14:44:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#STEP 0 確認有使用GPU
##記事本必須在GPU模式下運行，可以執行上方的nvidia-smi查看GPU是否有GPU。若没有，點擊左上角的 編輯>筆記本設定，把硬體加速器改成GPU。
##如記事本有出問題可在以下儲存庫回報
> https://github.com/ADT109119/LLaMA-Factory-Colab

In [2]:
#@title STEP 1 複製儲存庫並安裝必要的函式庫
#@markdown #STEP 1
#@markdown ##複製儲存庫並安裝必要的函式庫
#@markdown ##Clone repository & Install requirements lib

!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd ./LLaMA-Factory
!pip install -r requirements.txt
!pip install bitsandbytes>=0.39.0

#用比較殘暴的方式開啟 Gradio 的分享連結
train_web_py_file_path = "./src/train_web.py"
try:
  with open(train_web_py_file_path, 'r') as file:
    file_content = file.read()
  modified_content = file_content.replace("share=False", "share=True")
  with open(train_web_py_file_path, 'w') as file:
    file.write(modified_content)
except Exception as e:
    print(f'ERROR: {str(e)}')

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 22241, done.
remote: Total 22241 (delta 0), reused 0 (delta 0), pack-reused 22241 (from 1)
Receiving objects: 100% (22241/22241), 43.12 MiB | 15.55 MiB/s, done.
Resolving deltas: 100% (16230/16230), done.
/content/LLaMA-Factory
Ignoring transformers: markers 'python_version < "3.10"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title STEP 2 開啟WebUI
#@markdown #STEP 2
#@markdown ##開啟WebUI
#@markdown ##Run the WebUI

!CUDA_VISIBLE_DEVICES=0 python src/train_web.py

#STEP 3 上傳輸出的檔案到 HuggingFace


In [ ]:
#@title STEP 3.1 安裝HuggingFace API
#@markdown #STEP 3.1
#@markdown ##為了上傳模型至HuggingFace，需要先安裝HF的函式庫
#@markdown ##In order to upload the model to HuggingFace, we have to install the HF library first.
!pip install huggingface_hub

In [ ]:
#@title STEP 3.2 使用API Token登入
#@markdown #STEP 3.2
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title STEP 3.3 上傳模型
#@markdown #STEP 3.3
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login
from huggingface_hub import HfApi
api = HfApi()

model_id = "username/fine-tune-model" #@param {type:"string"}
export_folder_path = "/content/LLaMA-Factory/export" #@param {type:"string"}
api.create_repo(model_id, private=True, exist_ok=True, repo_type="model")
api.upload_folder(
    folder_path=export_folder_path,
    repo_id=model_id,
)

#STEP 4 保存到Google Drive(自選)

In [ ]:
#@title STEP 4.1 連接Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title STEP 4.2 複製檔案到Google Drive
Google_Drive_Folder = "/content/drive/MyDrive/LLaMA-Factory/" #@param {type:"string"}

!cp -rf /content/LLaMA-Factory/saves {Google_Drive_Folder}